In [ ]:
import tensorflow as tf
import numpy as np
import typing
import pandas as pd
import seaborn as sns
from tensorflow import keras
from hydra import compose, initialize
from omegaconf import OmegaConf
from musicnet.config.Config import Config
from musicnet.preprocessing.utils import get_datasets_info, load_vocabs
from musicnet.preprocessing.wav_chunks_tfrecord.utils import create_tf_record_ds
from musicnet.models.utils import MODEL_PATH
from musicnet.preprocessing.dataset.base import DsName
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
initialize(version_base=None, config_path="../scripts")
cfg = compose(config_name="defaults", overrides=["stages=[]", "exp=False"])

In [ ]:
config = typing.cast(Config, OmegaConf.to_object(cfg))
ds_infos = get_datasets_info(config)

datasets: dict[DsName, tf.data.Dataset] = {}
for ds_info in ds_infos:
    if ds_info.name in ["val", "test"]:
        datasets[ds_info.name] = create_tf_record_ds(ds_info.config, ds_info.name, shuffle=False)

model = keras.models.load_model(MODEL_PATH)

y_true = np.concatenate([y for x, y in datasets["test"]], axis=0)
y_pred = model.predict(datasets["test"])
y_pred = tf.sigmoid(y_pred)
y_pred = y_pred.numpy()

print("y_true shape", y_true.shape)
print("y_pred shape", y_pred.shape)


In [ ]:
# TODO: Establish what kind of metrics I want to have here:
# - bar plot of precision/f1 grouped by individual notes
# - bar plot of precision/f1 grouped by numer of silmulataneous notes
# - bar plot of precision/f1 grouped by note_length bins
# - bar plot of precision/f1 grouped by time_playing / time_remaining bins
# - If midi is a source: bar plot of precision/f1 grouped by velocity bins (like 30-40, 40-50, 50-60 etc.) 
# - visualization of some fragment of predictions vs true values (something like notebooks/utils:y_vs_y_pred_vis)

In [ ]:
y_true = y_true.reshape(-1, y_true.shape[-1])
y_pred = y_pred.reshape(-1, y_pred.shape[-1])

In [ ]:
instruments_vocab, notes_vocab = load_vocabs(config)

In [ ]:
notes_vocab

In [ ]:
def plot_f1_by_class(y_true: np.ndarray, y_pred: np.ndarray):
    f1_per_note = keras.metrics.F1Score(average=None, threshold=0.5)(y_true, y_pred).numpy()
    df = pd.DataFrame({
        "note": notes_vocab.keys(),
        "f1_score": f1_per_note
    })
    ax = sns.barplot(data=df, x="note", y="f1_score")
    ax.bar_label(ax.containers[0], fontsize=10)
    for bar in ax.patches:
        g = (bar.get_height() - min(f1_per_note)) / (max(f1_per_note) - min(f1_per_note))
        bar.set_color((1 - g, g, 0.))

In [ ]:
def plot_average_fp_fn_count_by_silmultaneous(y_true: np.ndarray, y_pred: np.ndarray):
    max_silmultaneous = int(np.max(y_true.sum(axis=1)))
    fps = []
    fns = []
    for s in range(0, max_silmultaneous + 1):
        indices = y_true.sum(axis=1) == s
        fps.append(((y_true[indices] == False) & (y_pred[indices] >= 0.5)).sum(axis=1).mean())
        fns.append(((y_true[indices] == True) & (y_pred[indices] < 0.5)).sum(axis=1).mean())
    df = pd.DataFrame({
        "silmultaneous_notes": list(range(0, max_silmultaneous + 1)) * 2,
        "metric": ["fp"] * len(fps) + ["fn"] * len(fns),
        "value": fps + fns
    })
    ax = sns.barplot(data=df, x="silmultaneous_notes", y="value", hue="metric")
    ax.bar_label(ax.containers[0], fontsize=10)
    ax.bar_label(ax.containers[1], fontsize=10)
    # for bar in ax.patches:
    #     g = (bar.get_height() - min(fps)) / (max(fps) - min(fps))
    #     bar.set_color((1 - g, g, 0.))

In [ ]:
def plot_confusion_matrix(y_true: np.ndarray, y_pred: np.ndarray):
    # For each note we calculate the percentage of false positives of other notes
    conf_matrix = []
    for n in notes_vocab.values():
        indices = y_true[:, n] == True
        false_positives = ((y_pred[indices] > 0.5) & (y_true[indices] == False)).sum(axis=0) / y_true.shape[0]
        conf_matrix.append(false_positives)
    conf_matrix = np.array(conf_matrix)
    disp = ConfusionMatrixDisplay(conf_matrix, display_labels=list(notes_vocab.keys()))
    plt.figure(figsize=(10, 10))
    plt.title("True labels vs false positives")
    disp.plot(ax=plt.gca())

In [ ]:
np.unique(y_true.sum(axis=1), return_counts=True)

In [ ]:
plot_f1_by_class(y_true, y_pred)

In [ ]:
plot_average_fp_fn_count_by_silmultaneous(y_true, y_pred)

In [ ]:
plot_confusion_matrix(y_true, y_pred)